In [ ]:
import os
import numpy as np
import time

from Util import my_helper
import copy
import itertools
import random
import pickle as cPickle
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python.layers import core as core_layers


from Model import LM
from Util.myAttWrapper import SelfAttWrapper
from Util import myResidualCell
from Util.bleu import BLEU
from Util.myUtil import *


tf.logging.set_verbosity(tf.logging.INFO)
sess_conf = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))




In [ ]:
def idx2str(idx):
    return " ".join(id2w[idxx] for idxx in idx)

def str2idx(idx):
    return [w2id[idxx] for idxx in idx]

def _init_data(name):
    w2id, id2w = cPickle.load(open('Data/%s/w2id_id2w.pkl' % name,'rb'))
    X_indices = cPickle.load(open('Data/%s/index.pkl' % name,'rb'))
    return X_indices, w2id, id2w

def _train_model(name, lr=10.0, l1_reg_lambda=0.00, l2_reg_lambda=0.00, close_loss_rate=0.00):
    qid_list = cPickle.load(open('Data/%s/qid_list.pkl'%name,'rb'))
    qid_list = [w2id[w] for w in qid_list]
    rnn_size = dict()
    rnn_size['Poem'] =  512
    rnn_size['Daily'] = 512 
    rnn_size['APRC'] = 1024
    
    num_layer = dict()
    num_layer['Poem'] = 2
    num_layer['Daily'] = 1
    num_layer['APRC'] = 1
    
    max_infer_length = dict()
    max_infer_length['Poem'] = 33
    max_infer_length['Daily'] = 50
    max_infer_length['APRC'] = 36
    
    model_iter = dict()
    model_iter['Poem'] = 30
    model_iter['Daily'] = 30 
    model_iter['APRC'] = 20
    
    assert name in ['Poem','Daily', 'APRC']

    BATCH_SIZE = 256
    NUM_EPOCH = 30
    train_dir ='Model/%s' % name
    dp = LM_DP(X_indices, w2id, BATCH_SIZE, n_epoch=NUM_EPOCH)
    g = tf.Graph() 
    sess = tf.Session(graph=g, config=sess_conf) 
    with sess.as_default():
        with sess.graph.as_default():
            model = LM(
                dp = dp,
                rnn_size = rnn_size[name],
                n_layers = num_layer[name],
                decoder_embedding_dim = rnn_size[name],
                cell_type='lstm',
                close_loss_rate = close_loss_rate,
                max_infer_length = max_infer_length[name],
                att_type='B',
                qid_list = qid_list,
                lr = lr,
                l1_reg_lambda = l1_reg_lambda,
                l2_reg_lambda = l2_reg_lambda,
                is_save = True,
                residual = True,
                is_jieba = False,
                sess=sess
            )


    util = LM_util(dp=dp, model=model)
    util.fit(train_dir=train_dir, is_bleu=False)
    return model

In [ ]:
task_name = 'APRC' # 'Daily', 'Poem'
assert task_name in ['Poem','Daily', 'APRC']
X_indices, w2id, id2w = _init_data(task_name)
model = _train_model(task_name, lr=10.0)

                